In [1]:
import pandas as pd
import numpy as np
import json
import re
from enum import Enum

In [2]:
# Create an ENUM
class Operation(Enum):

    SUM = "sum"

    SUB = "sub"

    MUL = "mul"
    
    DIV = "div"

In [3]:
"""
Retorna uma nova coluna com a soma das colunas pretendidas
numerator_c = ["ANO_EXERCICIO", "_5001_VENDAS_SERVICOS_PRESTADOS"]
numerator_operation = SUM, SUB, MUL
denominator_c = ["ANO_EXERCICIO", "_5001_VENDAS_SERVICOS_PRESTADOS"]
denominator_operation = SUM, SUB, MUL
c_name = NEW_COLUMN
"""
def div_columns(numerator_c,numerator_operation, denominator_c, denominator_operation, df):
    # Criar duas series (numerador e denominador)
    numerator_series = None
    denominator_series = None
    
    # Validar que tipo de operação numerador
    if numerator_operation == Operation.SUM:
        numerator_series = df.loc[:,numerator_c].sum(axis=1)
    elif numerator_operation == Operation.SUB:
        numerator_series = df[numerator_c[0]].subtract(df[numerator_c[1]])#df.loc[:,numerator_c].sub(axis=1)
    elif numerator_operation == Operation.MUL:
        numerator_series = df.loc[:,numerator_c].mul(axis=1)
    elif numerator_operation == Operation.DIV:
        numerator_series = df.loc[:,numerator_c].div(axis=1)
        
    # Validar que tipo de operação denominador
    if denominator_operation == Operation.SUM:
        denominator_series = df.loc[:,denominator_c].sum(axis=1)
    elif denominator_operation == Operation.SUB:
        denominator_series = df[denominator_c[0]].subtract(df[denominator_c[1]])#df.loc[:,denominator_c].sub(axis=1)
    elif denominator_operation == Operation.MUL:
        denominator_series = df.loc[:,denominator_c].mul(axis=1)
    elif denominator_operation == Operation.DIV:
        denominator_series = df.loc[:,denominator_c].div(axis=1)

    #df[c_name] = numerator_series / denominator_series
    return numerator_series / denominator_series

In [4]:
"""
Retorna uma nova coluna com a soma das colunas pretendidas
list_c = ["ANO_EXERCICIO", "_5001_VENDAS_SERVICOS_PRESTADOS"]
c_name = NEW_COLUMN
"""
def sum_columns(list_c, df):
    #df[c_name] = df.loc[:,list_c].sum(axis=1)
    return df.loc[:,list_c].sum(axis=1)

In [5]:
path_save = "../../Output/"

In [6]:
df = pd.read_csv(path_save + "finaldataset_non_filtered.csv", encoding="latin-1")

In [7]:
df.loc[df['Anulado'] == 1.0, ['N_Proj_anon', 'Motivo']]

,N_Proj_anon,Motivo
21,1025053,Incumprimento das obrigações ou resultados con...
22,1025053,Incumprimento das obrigações ou resultados con...
23,1025053,Incumprimento das obrigações ou resultados con...
24,1025053,Incumprimento das obrigações ou resultados con...
30,37858,"Insolvências, PER ..."
...,...,...
3026,5579015,Incumprimento das obrigações ou resultados con...
3027,2255722,Desistência do promotor
3028,2254854,Desistência do promotor
3029,2256032,Inexecução integral da candidatura


In [8]:
"""
Lista código com os rácios definidos no excel (Afonso Miao)
"""
# 5025_RESULTADO_LIQUIDO_PERIODO + 5018_GASTOS_DEPRECIACAO_AMORTIZA + (5147_PASSIVO_NC_TOTAL + 5159_PASSIVO_COR_TOTAL)
ebitda_formula = [
    "RESULTADO_LIQUIDO_PERIODO", "GASTOS_DEPRECIACAO_AMORTIZA", "PASSIVO_NC_TOTAL", "PASSIVO_COR_TOTAL"
]

# RESULTADO_LIQUIDO_PERIODO  + (PASSIVO_NC_TOTAL + PASSIVO_COR_TOTAL)
ebit_formula = [
    "RESULTADO_LIQUIDO_PERIODO", "PASSIVO_NC_TOTAL", "PASSIVO_COR_TOTAL"
]

#((ATIVO_NC_TOTAL + ATIVO_COR_TOTAL) + CP_TOTAL)
totalAssets_formula = [
    "ATIVO_NCOR_TOTAL",
    "ATIVO_COR_TOTAL"
]

#((PASSIVO_NC_TOTAL + PASSIVO_COR_TOTAL) + CP_TOTAL)
totalLiabilities_formula = [
    "PASSIVO_NC_TOTAL",
    "PASSIVO_COR_TOTAL"
]

#Total assets / total liabilities
total_assets_to_total_liabilities_num = ["Total_Assets"]
total_assets_to_total_liabilities_den = ["Total_Liabilities"]

#(ATIVO_COR_TOTAL - PASSIVO_COR_TOTAL) / Total assets
working_capital_divided_by_total_assets_num = ["ATIVO_COR_TOTAL", "PASSIVO_COR_TOTAL"]
working_capital_divided_by_total_assets_den = ["Total_Assets"]


#(Total assets - PASSIVO_COR_TOTAL) / ATIVO_NCOR_FIXOS_TANGIVEIS
capital_employed_to_net_fixed_assets_num = ["Total_Assets", "PASSIVO_COR_TOTAL"]
capital_employed_to_net_fixed_assets_den = ["ATIVO_NCOR_FIXOS_TANGIVEIS"]


#5017_ Resultado antes de depreciações, gastos de financiamento e impostos / 5001_VENDAS_SERVICOS_PRESTADOS
gross_profit_margin_num = ["RES_ANTES_DEPRECIACAO_GASTOS"]
gross_profit_margin_den = ["VENDAS_SERVICOS_PRESTADOS"]

#Total debt / total assets
total_debt_total_assets_num = ["Total_Liabilities"]
total_debt_total_assets_den = ["Total_Assets"]

#RESULTADO_LIQUIDO_PERIODO + GASTOS_DEPRECIACAO_AMORTIZACAO / PASSIVO_COR_TOTAL
operating_cash_flow_current_liabilities_num = ["RESULTADO_LIQUIDO_PERIODO", "GASTOS_DEPRECIACAO_AMORTIZA"]
operating_cash_flow_current_liabilities_den = ["PASSIVO_COR_TOTAL"]

#EBIT / Total assets
earnings_before_tax_and_interest_to_total_asset_num = ["EBIT"]
earnings_before_tax_and_interest_to_total_asset_den = ["Total_Assets"]


In [9]:
df["EBITDA"] = sum_columns(ebitda_formula, df)
df["EBIT"] = sum_columns(ebit_formula, df)
df["Total_Assets"] = sum_columns(totalAssets_formula, df)
df["Total_Liabilities"] = sum_columns(totalLiabilities_formula, df)
df["total_assets_to_total_liabilities"] = div_columns(total_assets_to_total_liabilities_num, Operation.SUM, total_assets_to_total_liabilities_den, Operation.SUM, df)
df["Working capital divided by total assets"] = div_columns(working_capital_divided_by_total_assets_num, Operation.SUB, working_capital_divided_by_total_assets_den, Operation.SUM, df)
df["Gross income divided by sales"] = div_columns(gross_profit_margin_num, Operation.SUM, gross_profit_margin_den, Operation.SUM, df)
df["total debt / total assets"] = div_columns(total_debt_total_assets_num, Operation.SUM, total_debt_total_assets_den, Operation.SUM, df)
df["earnings before tax and interest / total asset"] = div_columns(earnings_before_tax_and_interest_to_total_asset_num, Operation.SUM, earnings_before_tax_and_interest_to_total_asset_den, Operation.SUM, df)
df["Operating cash flow current liabilities"] = div_columns(operating_cash_flow_current_liabilities_num, Operation.SUM, operating_cash_flow_current_liabilities_den, Operation.SUM, df)

In [10]:
display(df)

,NIF_anon,ANO_EXERCICIO,DATA_RECEPCAO,VENDAS_SERVICOS_PRESTADOS,SUBSIDIOS_EXPLORACAO,GANHOS_PERDAS_SUBSIDIARIAS,VARIA_INVENTARIOS_PRODUCAO,TRABALHOS_PROPRIA_ENTIDADE,CMVMC,FSE,...,EBITDA,EBIT,Total_Assets,Total_Liabilities,total_assets_to_total_liabilities,Working capital divided by total assets,Gross income divided by sales,total debt / total assets,earnings before tax and interest / total asset,Operating cash flow current liabilities
0,533683393.0,2013,35:43.0,3575292.13,5899.36,0.00,9406.38,2060.62,1214731.77,888963.00,...,2032338.96,1741071.72,2820629.20,1673196.56,1.685773,-0.062573,0.117853,0.593200,0.617264,0.378150
1,833283384.0,2013,27:40.0,4112351.02,0.00,29583.09,151330.82,0.00,2228737.52,834301.58,...,1532453.15,1327973.55,3577302.24,1288560.85,2.776200,0.429615,0.077420,0.360205,0.371222,0.309288
2,833283384.0,2014,14:12.0,4375479.08,5363.90,7022.68,129606.23,0.00,2421525.79,847677.45,...,1702595.39,1500878.31,3784453.13,1489666.97,2.540469,0.406818,0.061806,0.393628,0.396591,0.230098
3,233783340.0,2013,40:46.0,2451333.02,24466.63,0.00,-4140.48,0.00,824628.05,824386.54,...,550278.38,475503.56,1659234.14,472313.17,3.512996,0.621067,0.034758,0.284657,0.286580,0.173488
4,233783340.0,2014,42:30.0,2858042.68,31100.02,0.00,-48818.66,0.00,790521.13,997313.78,...,539963.26,454856.61,1636920.06,415502.37,3.939617,0.658108,0.053382,0.253832,0.277873,0.306798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3026,981689435.0,2020,11:24.0,0.00,4358.00,0.00,0.00,0.00,5230.93,21434.80,...,283023.68,283023.68,463023.68,344436.63,1.344293,0.093513,-inf,0.743886,0.611251,-0.275711
3027,281789489.0,2020,44:50.0,67374.35,0.00,0.00,0.00,0.00,52468.25,7351.29,...,43591.31,42847.56,92847.56,40911.47,2.269475,0.328018,0.053758,0.440631,0.461483,0.065503
3028,382488479.0,2020,36:43.0,0.00,0.00,0.00,0.00,0.00,0.00,4611.50,...,-5478.19,-5478.19,44521.81,1694.00,26.282060,0.961951,-inf,0.038049,-0.123045,-4.233878
3029,82188456.0,2020,57:59.0,24953.19,0.00,0.00,0.00,0.00,0.00,7874.25,...,7283.95,7283.95,7283.95,2318.02,3.142315,0.681763,0.228695,0.318237,1.000000,2.142315


In [11]:
def calculate_average_ratios(number_days, ratio):
    if(ratio == 0):
        return None
    else:
        return number_days/ratio

In [12]:
#5001_VENDAS_SERVICOS_PRESTADOS / 5115_ATIVO_COR_CLIENTES
# accounts_receivable_turnover_num = ["VENDAS_SERVICOS_PRESTADOS"]
# accounts_receivable_turnover_den = ["ATIVO_COR_CLIENTES"]

#365 days / Receivables turnover ratio --> Average collection period for receivables


accounts_receivable_turnover_num = ["VENDAS_SERVICOS_PRESTADOS"]
accounts_receivable_turnover_den = ["ATIVO_COR_CLIENTES"]



#(5115_ATIVO_COR_CLIENTES / 5001_VENDAS_SERVICOS_PRESTADOS) --> Creditors turnover
#365 days / Creditors turnover --> Average payment period to creditors
creditors_turnover_num = ['ATIVO_COR_CLIENTES']
creditors_turnover_den = ['VENDAS_SERVICOS_PRESTADOS']


# CMVMC Inventário do Promotor em t-1 = prom_cmvmc/ATIVO_COR_INVENTARIOS --> Inventory turnover
# 365 days / (Inventory turnover ratio) = #CMVMC Inventário do Promotor em t-1 = prom_cmvmc/inventario_t-2
inventory_turnover_num = ['CMVMC']
inventory_turnover_den = ['ATIVO_COR_INVENTARIOS']

In [13]:
df["Accounts_Receivables_Turnover"] = div_columns(accounts_receivable_turnover_num, Operation.SUM, accounts_receivable_turnover_den, Operation.SUM, df)
df["Creditors_Turnover"] = div_columns(creditors_turnover_num, Operation.SUM, creditors_turnover_den, Operation.SUM, df)
df["Inventory_Turnover"] = div_columns(inventory_turnover_num, Operation.SUM, inventory_turnover_den, Operation.SUM, df)


In [14]:
df['Average_Collection_Period_For_Receivables'] = df.apply(lambda x: calculate_average_ratios(365, x['Accounts_Receivables_Turnover']), axis=1)
df['Average_Payment_Period_To_Creditors'] = df.apply(lambda x: calculate_average_ratios(365, x['Creditors_Turnover']), axis=1)
df['Average_Turnover_Period_For_Inventories'] = df.apply(lambda x: calculate_average_ratios(365, x['Inventory_Turnover']), axis=1)

In [15]:
df['ANO_EXERCICIO']

0       2013
1       2013
2       2014
3       2013
4       2014
        ... 
3026    2020
3027    2020
3028    2020
3029    2020
3030    2020
Name: ANO_EXERCICIO, Length: 3031, dtype: int64

In [16]:
"""
3 CENARIOS:
year_rollback = 1 (T-1)
year_rollback = 2 (T-2)
year_rollback = 3 (T-3)

1) calcular o ano anterior consoante years_to_rollback
2) validar se existe esse ano na lista: caso não --> None
3) Caso exista, ir buscar o valor do rácio para esse ano
    a) ratio_rollback = df.loc[df['ANO_EXERCICIO'] == year_rollback, [ratio]]
    b) ratio_value = (actual_ratio_value - rollback_ratio_value) / rollback_ratio_value * 100

"""
#(VENDAS_SERVICOS_PRESTADOS - VENDAS_SERVICOS_PRESTADOS_ANOS_ANTERIORES) / VENDAS_SERVICOS_PRESTADOS_ANOS_ANTERIORES x 100
def calculate_rollback_ratios(years_to_rollback, ratio_column, df, current_row):
    year = current_row['ANO_EXERCICIO']
    #print('Year: ', year)
    year_rollback = year - years_to_rollback
    #print('Year rollback: ', year_rollback)
    list_available_years = current_row['LISTA_ANO_EXERCICIO']
     # Extract the elements of the list from the string
    elements = re.findall(r'\d+', list_available_years)
    # Convert the elements to integers
    list_available_years_oficial = [int(x) for x in elements]
    #print('List years: ', list_available_years_oficial)
    #print('List years type: ', type(list_available_years_oficial))
    if int(year_rollback) in list_available_years_oficial:
        #Calculate the ratio
        n_project = current_row['N_Proj_anon']
        current_value = current_row[ratio_column]
        rollback_value = df.loc[(df['ANO_EXERCICIO'] == year_rollback) & (df['N_Proj_anon'] == n_project), [ratio_column]].values[0][0]
        # print("Current value: ", current_value)
        # print("Type: ", type(current_value))
        # print("Rollback value: ", rollback_value)
        # print("Type: ", type(rollback_value))
        ratio_value = (current_value - rollback_value) / current_value * 100
        return ratio_value
    else:
        return None

In [17]:
df['Growth_Rate_Net_Sales_T1'] = df.apply(lambda x: calculate_rollback_ratios(1, 'VENDAS_SERVICOS_PRESTADOS', df, x), axis=1)
df['Growth_Rate_Net_Sales_T2'] = df.apply(lambda x: calculate_rollback_ratios(2, 'VENDAS_SERVICOS_PRESTADOS', df, x), axis=1)
df['Growth_Rate_Net_Sales_T3'] = df.apply(lambda x: calculate_rollback_ratios(3, 'VENDAS_SERVICOS_PRESTADOS', df, x), axis=1)

C:\Users\afons\AppData\Local\Temp\ipykernel_11916\4267884457.py:36: RuntimeWarning: invalid value encountered in scalar divide
  ratio_value = (current_value - rollback_value) / current_value * 100
C:\Users\afons\AppData\Local\Temp\ipykernel_11916\4267884457.py:36: RuntimeWarning: invalid value encountered in scalar divide
  ratio_value = (current_value - rollback_value) / current_value * 100
C:\Users\afons\AppData\Local\Temp\ipykernel_11916\4267884457.py:36: RuntimeWarning: invalid value encountered in scalar divide
  ratio_value = (current_value - rollback_value) / current_value * 100
C:\Users\afons\AppData\Local\Temp\ipykernel_11916\4267884457.py:36: RuntimeWarning: invalid value encountered in scalar divide
  ratio_value = (current_value - rollback_value) / current_value * 100
C:\Users\afons\AppData\Local\Temp\ipykernel_11916\4267884457.py:36: RuntimeWarning: invalid value encountered in scalar divide
  ratio_value = (current_value - rollback_value) / current_value * 100
C:\Users\a

In [18]:
df['Growth_Rate_Total_Assets_T1'] = df.apply(lambda x: calculate_rollback_ratios(1, 'Total_Assets', df, x), axis=1)
df['Growth_Rate_Total_Assets_T2'] = df.apply(lambda x: calculate_rollback_ratios(2, 'Total_Assets', df, x), axis=1)
df['Growth_Rate_Total_Assets_T3'] = df.apply(lambda x: calculate_rollback_ratios(3, 'Total_Assets', df, x), axis=1)

In [19]:
df['Parametros/Ano_Cand']

0       2014.0
1       2015.0
2       2015.0
3       2015.0
4       2015.0
         ...  
3026    2021.0
3027    2019.0
3028    2019.0
3029    2019.0
3030    2019.0
Name: Parametros/Ano_Cand, Length: 3031, dtype: float64

In [20]:
# list_ids = [
# 18917,
# 20870,
# 20932,
# 23319,
# 26698,
# 30449,
# 30604,
# 33921,
# 35285,
# 280681,
# 409176,
# 449383,
# 455335,
# 455738,
# 456451,
# 458931,
# 460140,
# 460295,
# 461473,
# 462341,
# 462930,
# 463085,
# 464325,
# 464573,
# 465999,
# 466402,
# 467828,
# 470308,
# 471672,
# 472819,
# 473935,
# 473966,
# 476849,
# 478089,
# 483359,
# 551001,
# 553822,
# 554194,
# 555558,
# 556953,
# 564207,
# 572019,
# 572081,
# 576390,
# 579025,
# 579242,
# 582528,
# 583675,
# 584047,
# 584481,
# 585349,
# 586620,
# 587612,
# 589689,
# 590340,
# 590526,
# 594773,
# 596509,
# 599671,
# 599981,
# 605809,
# 605902,
# 608940,
# 609684,
# 609746,
# 610552,
# 611451,
# 612319,
# 612536,
# 612908,
# 613714,
# 614458,
# 614582,
# 616163,
# 617093,
# 617403,
# 617775,
# 620286,
# 706249,
# 712077,
# 712573,
# 714588,
# 715301,
# 716479,
# 719610,
# 720075,
# 726492,
# 727856,
# 730181,
# 730336,
# 731390,
# 732785,
# 734211,
# 736381,
# 737342,
# 740752,
# 740783,
# 741527,
# 741806,
# 742798,
# 743232,
# 748130,
# 748192,
# 749711,
# 753059,
# 753524,
# 756283,
# 757554,
# 758887,
# 760437,
# 762235,
# 864318,
# 1007259,
# 1025177,
# 1025456,
# 1028587,
# 1035407,
# 1036120,
# 1036337,
# 1038538,
# 1038972,
# 1040615,
# 1043715,
# 1046877,
# 1047187,
# 1048489,
# 1053883,
# 1054441,
# 1059029,
# 1059277,
# 1059525,
# 1065291,
# 1197072,
# 1197103,
# 1198963,
# 1234489,
# 1234551,
# 1236225,
# 1237248,
# 1237682,
# 1239263,
# 1240100,
# 1240317,
# 1240720,
# 1241526,
# 1253709,
# 1442344,
# 1443894,
# 1446374,
# 1447955,
# 1448079,
# 1454682,
# 1458278,
# 1512466,
# 2194032,
# 2206711,
# 2254854,
# 2255722,
# 2256032,
# ]

In [21]:
# df_teste1 = df.loc[df['ANO_EXERCICIO'] == (df['Parametros/Ano_Cand'] - 1)]
# df_teste2 = df.loc[df['ANO_EXERCICIO'] == (df['Parametros/Ano_Cand'])]

In [22]:
# df_teste1[df_teste1['N_Proj_anon'].isin(list_ids)]

In [23]:
# df_teste2.shape

In [24]:
# df_teste2.shape

In [25]:
# df_teste2[df_teste2['N_Proj_anon'] == 18917]

In [26]:
# df_teste3 = df_teste1.merge(df_teste2, how="inner", on="N_Proj_anon")

In [27]:
# df_teste3

In [28]:
df.columns.tolist()

['NIF_anon',
 'ANO_EXERCICIO',
 'DATA_RECEPCAO',
 'VENDAS_SERVICOS_PRESTADOS',
 'SUBSIDIOS_EXPLORACAO',
 'GANHOS_PERDAS_SUBSIDIARIAS',
 'VARIA_INVENTARIOS_PRODUCAO',
 'TRABALHOS_PROPRIA_ENTIDADE',
 'CMVMC',
 'FSE',
 'GASTOS_PESSOAL',
 'IMPARIDADE_INVENTARIOS',
 'IMPARIDADE_DIVIDAS_A_RECEBER',
 'PROVISOES',
 'IMPARIDADE_INVENTA_N_AMORT_INVESTIMENTOS_N_DEPRECIAVEIS',
 'OUTRAS_IMPARIDADES',
 'AUMENTOS_RED_JUSTO_VALOR',
 'OUTROS_REDIMENTOS_GANHOS',
 'OUTROS_GASTOS_PERDAS',
 'RES_ANTES_DEPRECIACAO_GASTOS',
 'GASTOS_DEPRECIACAO_AMORTIZA',
 'IMPARIDADE_INVEST_AMORTIZ_DEPRECIAVEIS',
 'RESULTADO_OPERACIONAL',
 'JUROS_RENDIME_SIMILARES_OBT',
 'JUROS_GAST_SIMILARES_SUPORT',
 'RES_ANTES_IMPOSTOS',
 'IMPOSTO_RENDIMENTO_PERIODO',
 'RESULTADO_LIQUIDO_PERIODO',
 'ATIVO_NCOR_FIXOS_TANGIVEIS',
 'ATIVO_NCOR_PROPRI_INVESTIMENTO',
 'ATIVO_NCOR_GOODWILL',
 'ATIVO_NCOR_INTANGIVEIS',
 'ATIVO_NCOR_BIOLOGICOS',
 'ATIVO_NCOR_PART_FINAN_EQV_PAT',
 'ATIVO_NCOR_PART_FIN_OUTROS_MET',
 'ATIVO_NCOR_ACCIONISTAS_SOCIOS'

In [29]:
racios_not_remove = [
    'NIF_anon', 
'ANO_EXERCICIO',  
'DATA_RECEPCAO', 
'N_Proj_anon', 
'Dimensao_Empresa', 
'Nproj_anon_x', 
'Motivo', 
'Anulado', 
'Nproj_anon_y', 
'Tipo', 
'Terminado', 
'LISTA_ANO_EXERCICIO', 
'ANO_EXERCICIO_VALIDOS', 
'ANO_EXERCICIO_VALIDOS_ORIGINAL', 
'Projeto_Valido', 
]

In [30]:
lista_racios = [] #df.columns.tolist().remove(racios_not_remove)

In [31]:
for racio in df.columns.tolist():
    if racio not in racios_not_remove:
        lista_racios.append(racio)

In [32]:
df_projetos_validos = df.loc[df['Projeto_Valido'] == 1]
df_projetos_invalidos = df.loc[df['Projeto_Valido'] == 0]

In [33]:
df_projetos_invalidos[lista_racios] = 0

C:\Users\afons\AppData\Local\Temp\ipykernel_11916\2102159377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_projetos_invalidos[lista_racios] = 0


In [34]:
df_projetos_invalidos

,NIF_anon,ANO_EXERCICIO,DATA_RECEPCAO,VENDAS_SERVICOS_PRESTADOS,SUBSIDIOS_EXPLORACAO,GANHOS_PERDAS_SUBSIDIARIAS,VARIA_INVENTARIOS_PRODUCAO,TRABALHOS_PROPRIA_ENTIDADE,CMVMC,FSE,...,Inventory_Turnover,Average_Collection_Period_For_Receivables,Average_Payment_Period_To_Creditors,Average_Turnover_Period_For_Inventories,Growth_Rate_Net_Sales_T1,Growth_Rate_Net_Sales_T2,Growth_Rate_Net_Sales_T3,Growth_Rate_Total_Assets_T1,Growth_Rate_Total_Assets_T2,Growth_Rate_Total_Assets_T3
2712,138489425.0,2016,51:55.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2766,231189452.0,2015,35:23.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2784,832281467.0,2014,39:21.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2802,364389458.0,2014,18:57.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2803,665183418.0,2014,42:24.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,80182420.0,2020,17:01.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3027,281789489.0,2020,44:50.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3028,382488479.0,2020,36:43.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3029,82188456.0,2020,57:59.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
df_etl_final1 = df_projetos_validos.loc[df['ANO_EXERCICIO'] == (df['Parametros/Ano_Cand'] - 1)]
df_etl_final2= df_projetos_invalidos
df_etl_final = pd.concat([df_etl_final1, df_etl_final2])
df_etl_final

,NIF_anon,ANO_EXERCICIO,DATA_RECEPCAO,VENDAS_SERVICOS_PRESTADOS,SUBSIDIOS_EXPLORACAO,GANHOS_PERDAS_SUBSIDIARIAS,VARIA_INVENTARIOS_PRODUCAO,TRABALHOS_PROPRIA_ENTIDADE,CMVMC,FSE,...,Inventory_Turnover,Average_Collection_Period_For_Receivables,Average_Payment_Period_To_Creditors,Average_Turnover_Period_For_Inventories,Growth_Rate_Net_Sales_T1,Growth_Rate_Net_Sales_T2,Growth_Rate_Net_Sales_T3,Growth_Rate_Total_Assets_T1,Growth_Rate_Total_Assets_T2,Growth_Rate_Total_Assets_T3
0,533683393.0,2013,35:43.0,3575292.13,5899.36,0.00,9406.38,2060.62,1214731.77,888963.00,...,5.841945,37.339044,3567.980995,62.479196,NaN,NaN,NaN,NaN,NaN,NaN
2,833283384.0,2014,14:12.0,4375479.08,5363.90,7022.68,129606.23,0.00,2421525.79,847677.45,...,3.230099,133.487586,998.032883,112.999628,6.013697,NaN,NaN,5.473734,NaN,NaN
4,233783340.0,2014,42:30.0,2858042.68,31100.02,0.00,-48818.66,0.00,790521.13,997313.78,...,2.058681,26.200614,5084.804512,177.297966,14.230356,NaN,NaN,-1.363175,NaN,NaN
5,533884363.0,2013,04:47.0,3104380.30,23179.61,10.12,839.00,0.00,695060.18,745809.78,...,15.140654,96.597469,1379.176921,24.107281,NaN,NaN,NaN,NaN,NaN,NaN
8,633484313.0,2015,46:12.0,19657707.01,0.00,1261076.94,-361674.88,0.00,13328318.33,1715405.80,...,17.118323,204.857985,650.328567,21.322182,7.824969,-0.12517,NaN,1.946931,5.088581,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,80182420.0,2020,17:01.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0
3027,281789489.0,2020,44:50.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0
3028,382488479.0,2020,36:43.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0
3029,82188456.0,2020,57:59.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0


In [36]:
df_etl_final[df_etl_final['Terminado'] == 1.0]

,NIF_anon,ANO_EXERCICIO,DATA_RECEPCAO,VENDAS_SERVICOS_PRESTADOS,SUBSIDIOS_EXPLORACAO,GANHOS_PERDAS_SUBSIDIARIAS,VARIA_INVENTARIOS_PRODUCAO,TRABALHOS_PROPRIA_ENTIDADE,CMVMC,FSE,...,Inventory_Turnover,Average_Collection_Period_For_Receivables,Average_Payment_Period_To_Creditors,Average_Turnover_Period_For_Inventories,Growth_Rate_Net_Sales_T1,Growth_Rate_Net_Sales_T2,Growth_Rate_Net_Sales_T3,Growth_Rate_Total_Assets_T1,Growth_Rate_Total_Assets_T2,Growth_Rate_Total_Assets_T3
0,533683393.0,2013,35:43.0,3575292.13,5899.36,0.00,9406.38,2060.62,1214731.77,888963.00,...,5.841945,37.339044,3567.980995,62.479196,NaN,NaN,NaN,NaN,NaN,NaN
2,833283384.0,2014,14:12.0,4375479.08,5363.90,7022.68,129606.23,0.00,2421525.79,847677.45,...,3.230099,133.487586,998.032883,112.999628,6.013697,NaN,NaN,5.473734,NaN,NaN
4,233783340.0,2014,42:30.0,2858042.68,31100.02,0.00,-48818.66,0.00,790521.13,997313.78,...,2.058681,26.200614,5084.804512,177.297966,14.230356,NaN,NaN,-1.363175,NaN,NaN
5,533884363.0,2013,04:47.0,3104380.30,23179.61,10.12,839.00,0.00,695060.18,745809.78,...,15.140654,96.597469,1379.176921,24.107281,NaN,NaN,NaN,NaN,NaN,NaN
8,633484313.0,2015,46:12.0,19657707.01,0.00,1261076.94,-361674.88,0.00,13328318.33,1715405.80,...,17.118323,204.857985,650.328567,21.322182,7.824969,-0.12517,NaN,1.946931,5.088581,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2968,274683414.0,2016,32:49.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0
2973,874384403.0,2016,00:43.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0
2980,275484418.0,2017,09:24.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0
2992,577485425.0,2017,21:48.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0


In [37]:
len(df.loc[df['Terminado'] == 1.0, ['N_Proj_anon']]['N_Proj_anon'].unique())

805

In [38]:
df_etl_final[df_etl_final['Anulado'] == 1.0]

,NIF_anon,ANO_EXERCICIO,DATA_RECEPCAO,VENDAS_SERVICOS_PRESTADOS,SUBSIDIOS_EXPLORACAO,GANHOS_PERDAS_SUBSIDIARIAS,VARIA_INVENTARIOS_PRODUCAO,TRABALHOS_PROPRIA_ENTIDADE,CMVMC,FSE,...,Inventory_Turnover,Average_Collection_Period_For_Receivables,Average_Payment_Period_To_Creditors,Average_Turnover_Period_For_Inventories,Growth_Rate_Net_Sales_T1,Growth_Rate_Net_Sales_T2,Growth_Rate_Net_Sales_T3,Growth_Rate_Total_Assets_T1,Growth_Rate_Total_Assets_T2,Growth_Rate_Total_Assets_T3
24,833688307.0,2016,47:20.0,7363895.17,37205.08,0.0,-2367911.54,0.0,1364438.79,1414833.45,...,0.484667,109.097631,1221.153919,753.094799,-2.682106,-11.053803,-8.232199,4.185558,4.371426,4.837481
30,333381322.0,2013,00:21.0,2469363.43,0.00,0.0,-44564.00,0.0,1372077.34,721653.82,...,1.689593,491.584525,271.011379,216.028367,NaN,NaN,NaN,NaN,NaN,NaN
32,433881397.0,2014,08:18.0,2078133.36,0.00,0.0,-2000.00,0.0,246333.04,372290.59,...,0.993278,226.724159,587.608310,367.469991,-26.390549,NaN,NaN,12.742849,NaN,NaN
38,533181305.0,2014,21:18.0,788601.20,17268.43,0.0,104208.83,0.0,310830.99,147081.52,...,1.773267,74.174978,1796.090862,205.834767,-2.484521,NaN,NaN,6.283814,NaN,NaN
44,633882356.0,2018,40:34.0,14481549.67,0.00,0.0,-44507.36,0.0,9408127.47,1774013.66,...,1.783396,138.312026,963.220650,204.665696,21.670485,27.043427,24.709775,7.862315,0.565992,4.549447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,80182420.0,2020,17:01.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3027,281789489.0,2020,44:50.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3028,382488479.0,2020,36:43.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3029,82188456.0,2020,57:59.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [39]:
len(df.loc[df['Anulado'] == 1.0, ['N_Proj_anon']]['N_Proj_anon'].unique())

552

In [40]:
df_etl_final.to_csv(path_save + 'finaldataset_for_ML2.csv', index=False, encoding='latin-1')

In [41]:
assert(False)

AssertionError: 

In [ ]:
df_new = pd.read_csv(path_save + 'finaldataset_for_ML2.csv', encoding='latin-1')

In [ ]:
df_old = pd.read_csv(path_save + 'finaldataset_for_ML.csv', encoding='latin-1')

In [ ]:
def check_project_year_candidate_duplicates(df, columns=['N_Proj_anon', 'ANO_EXERCICIO']):
    new_df = df.groupby(columns).size()
    new_df = new_df.to_frame(name = 'size').reset_index()
    display(new_df[new_df['size']>1])

In [ ]:
check_project_year_candidate_duplicates(df_new)

,N_Proj_anon,ANO_EXERCICIO,size


In [ ]:
df_new.shape

(1356, 437)

In [ ]:
len(df_new['N_Proj_anon'].unique())

1356

In [ ]:
len(df_old['N_Proj_anon'].unique())

1197

In [ ]:
Esta se a perder projetos por causa do filtro do param
Tem de existir um mecanismo para isolar os projetos invalidos (criar uma nova coluna no notebook filter all projects final dataset com flag valido)
E depois no final fazer union